In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from autocorrect import spell
from textblob import TextBlob

C:\Users\Lenovo\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
train = pd.read_csv("train.csv")
print (train.shape)
train.head(10)

(38932, 5)


,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy
5,id10331,We had - rooms. One was very nice and clearly ...,InternetExplorer,Desktop,happy
6,id10332,My husband and I have stayed in this hotel a f...,Firefox,Tablet,not happy
7,id10333,My wife & I stayed in this glorious city a whi...,Google Chrome,Mobile,happy
8,id10334,My boyfriend and I stayed at the Fairmont on a...,Internet Explorer,Desktop,happy
9,id10335,"Wonderful staff, great location, but it was de...",Chrome,Tablet,not happy


In [3]:
test = pd.read_csv("test.csv")
print (test.shape)
test.head()

(29404, 4)


,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet
3,id80135,This hotel location is excellent and the rooms...,Edge,Mobile
4,id80136,This hotel is awesome I love the service Antho...,Mozilla,Mobile


In [8]:
stop = set(stopwords.words("english"))
#stm = PorterStemmer()
Lemma = WordNetLemmatizer()
def clean_data(txt):
    txt = str(txt)
    txt = re.sub("[^a-zA-Z]", " ", txt) 
#     txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
#     txt = re.sub(r'\n',r' ',txt)
    b = TextBlob(txt)
    txt = b.correct()
    txt = txt.lower().split()  
    txt = [w for w in txt if not w in stop]  
    txt = [Lemma.lemmatize(x) for x in txt]
    txt = " ".join(txt)
    return txt

In [9]:
test["Is_Response"]  = np.nan
alldata = pd.concat([train,test]).reset_index(drop = True)
print (alldata.shape)
alldata.head()

(68336, 5)


,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [ ]:
le = LabelEncoder()
col = ["Browser_Used", "Device_Used"]
for x in col:
    alldata[x] = le.fit_transform(alldata[x])
alldata.Description = [clean_data(x) for x in alldata.Description]
alldata.head()

In [ ]:
# here we tried to do with sentimental analysis but it didnt worked well.
# sentiment = SentimentIntensityAnalyzer()
# sample_review = alldata.Description[:10]
# for sentences in sample_review:
#     sentences
#     ss = sentiment.polarity_scores(sentences)
#     for k in sorted(ss):
#         print('{0}: {1}, '.format(k, ss[k]))
#     print(sentences)

## countvectorizer

In [15]:
count = CountVectorizer(analyzer = "word", ngram_range = (1,1))
count = count.fit_transform(alldata.Description)

In [16]:
count.get_feature_names()## to get all words in a bag of words

AttributeError: get_feature_names not found

In [ ]:
count.shape

In [ ]:
pca2 = TruncatedSVD(1000,random_state = 0)
pca_new2 = pca2.fit_transform(count)

In [ ]:
pca2.explained_variance_ratio_.sum()

In [ ]:
pca_df2 = pd.DataFrame(pca_new2)
pca_df2.shape

In [ ]:
count_df = pd.concat([alldata[col],pca_df2],axis = 1).reset_index(drop = True)


In [ ]:
print count_df.shape
count_df.head()

In [ ]:
target2 = alldata.Is_Response[alldata.Is_Response.notnull()]
target2[target2 =="happy"] = 1
target2[target2 =="not happy"] = 0
print len(target2)
train_df2 = count_df[:len(train)]
print len(train_df2)
test_df2 = count_df[len(train):]

In [ ]:
[list(x) for x in alldata]

In [ ]:
import lightgbm as lgb

In [ ]:
d_train2 = lgb.Dataset(train_df2, label = target2)

In [ ]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 7, 
    'num_leaves': 21, 
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [ ]:
lgb_cv2 = lgb.cv(params, d_train2, num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
nround2 = lgb_cv2['binary_error-mean'].index(np.min(lgb_cv2['binary_error-mean']))

In [ ]:
model2 = lgb.train(params, d_train2, num_boost_round=nround2)
model2

In [ ]:
preds2 = model.predict(test_df2)

In [ ]:
model2.feature_importance()

In [ ]:
def to_labels(x):
    if x > 0.66:  # cutoff - you can change it and see if accuracy improves or plot AUC curve. 
        return "happy"
    return "not_happy"

sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds2})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))
sub3 = sub3[['User_ID','Is_Response']]
sub3.to_csv('sub3(1)_lgb.csv', index=False) # 0.414

## tfidf

In [ ]:
tfid = TfidfVectorizer(analyzer = "word",ngram_range=(1,1))
tfid = tfid.fit_transform(alldata.Description)

In [ ]:
# tfid = pd.DataFrame(tfid.todense())
tfid.shape

In [ ]:
pca = TruncatedSVD(1000,random_state = 0)
pca_new = pca.fit_transform(tfid)

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
2+5

In [ ]:
pca_df = pd.DataFrame(pca_new)
pca_df.shape

In [ ]:
tfid_df = pd.concat([alldata[col],pca_df],axis = 1).reset_index(drop = True)


In [ ]:
print tfid_df.shape
tfid_df.head()

In [ ]:
target = alldata.Is_Response[alldata.Is_Response.notnull()]
target[target =="happy"] = 1
target[target =="not happy"] = 0
print len(target)
train_df = tfid_df[:len(train)]
print len(train_df)
test_df = tfid_df[len(train):]

In [19]:
import lightgbm as lgb

In [20]:
d_train = lgb.Dataset(train_df, label = target)

NameError: name 'train_df' is not defined

In [ ]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 7, 
    'num_leaves': 21, 
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [ ]:
lgb_cv = lgb.cv(params, d_train, num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [ ]:
model = lgb.train(params, d_train, num_boost_round=nround)
model

In [ ]:
preds = model.predict(test_df)

In [ ]:
model.feature_importance()

In [ ]:
test_df.shape

In [ ]:
def to_labels(x):
    if x > 0.66:  # cutoff - you can change it and see if accuracy improves or plot AUC curve. 
        return "happy"
    return "not_happy"

sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))
sub3 = sub3[['User_ID','Is_Response']]
sub3.to_csv('sub3_lgb.csv', index=False) # 0.87306

In [ ]:
2+5

## tfidf with direct 1500 words without pca

In [ ]:
tfid = TfidfVectorizer(analyzer = "word",ngram_range=(1,1), max_features=1500)
tfid = tfid.fit_transform(alldata.Description)

In [ ]:
tfid_df = pd.concat([alldata[col],pd.DataFrame(tfid.todense())],axis = 1).reset_index(drop = True)


In [ ]:
print (tfid_df.shape)

tfid_df.head()

In [ ]:
target = alldata.Is_Response[alldata.Is_Response.notnull()]
target[target =="happy"] = 1
target[target =="not happy"] = 0
print (len(target))
train_df = tfid_df[:len(train)]
print (len(train_df))
test_df = tfid_df[len(train):]

In [ ]:
import lightgbm as lgb

In [ ]:
d_train = lgb.Dataset(train_df, label = target)

In [ ]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.04, 
    'max_depth': 7, 
    'num_leaves': 1000, 
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [ ]:
lgb_cv = lgb.cv(params, d_train, num_boost_round=1000, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

In [ ]:
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [ ]:
model = lgb.train(params, d_train, num_boost_round=nround)
model

In [ ]:
preds = model.predict(test_df)

In [ ]:
preds

In [ ]:
model.feature_importance()

In [ ]:
# from sklearn.metric import confusion_matrix
# confusion_matrix()

In [ ]:
def to_labels(x):
    if x > 0.55:  # cutoff - you can change it and see if accuracy improves or plot AUC curve. #ideally found is 0.55 fo which it was .88117
        return "happy"
    return "not_happy"

sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))
sub3 = sub3[['User_ID','Is_Response']]
sub3.to_csv('sub3_lgb.csv', index=False) # 0.88117

## Random forest

In [84]:
target.as_matrix()

array([0, 0, 0, ..., 0, 0, 1], dtype=object)

In [63]:
from sklearn.model_selection import GridSearchCV
def cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=5):
    gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(Xtrain, ytrain)
    print ("BEST PARAMS", gs.best_params_)
    best = gs.best_estimator_
    return best

In [ ]:
from sklearn.cross_validation import train_test_split
def do_classify(clf, parameters, Xtrain,ytrain,Xtest ,standardize=False):
#     subdf=x_df
#     if standardize:
#         subdfstd=(subdf - subdf.mean())/subdf.std()
#     else:
#         subdfstd=subdf
#     X=subdfstd.values
#     y=y_df.values
#     Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size,random_state = 0)
    clf = cv_optimize(clf, parameters, Xtrain.values, list(ytrain.values))
    clf.fit(Xtrain.values, ytrain.values)
    pred = clf.predict(Xtest.values)
#     training_accuracy = clf.score(Xtrain, ytrain)
#     test_accuracy = clf.score(Xtest, ytest)
#     print ("Accuracy on training data: %0.2f" % (training_accuracy))
#     print ("Accuracy on test data:     %0.2f" % (test_accuracy))
    return pred

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pred = do_classify(RandomForestClassifier(random_state = 0),{"min_samples_split":range(2,25),"n_estimators":range(100,500,100)},train_df,target,test_df)
pred

In [ ]:
def to_labels(x):
    if x > 0.55:  # cutoff - you can change it and see if accuracy improves or plot AUC curve. #ideally found is 0.55 fo which it was .88117
        return "happy"
    return "not_happy"

sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':pred})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))
sub3 = sub3[['User_ID','Is_Response']]
sub3.to_csv('sub3_lgb.csv', index=False) # 0.88117